In [21]:
import nltk
import string
import numpy as np
import pandas as pd
import os
from nltk.corpus import stopwords


from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, GlobalMaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding
from keras.utils import to_categorical

In [2]:
df = pd.read_csv('timesofindia_train.csv', sep='\t')

In [3]:
df_sent = (df['sentence'].str.len() > 10)
ne_df = df.loc[df_sent]
df = ne_df
df = df[~df.sentence.str.startswith('https')]
df = df[~df.sentence.str.startswith('url : ')]

In [4]:
sent_list = []
label_list = []

sent_list = df["sentence"].tolist()
label_list = df["etype"].tolist()

print(len(label_list))
print(label_list[0])

1323
0


In [5]:
def word_embeddings(path):
    embeddings_index = dict()
    f = open(path)
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    f.close()
    return embeddings_index

In [6]:
glove_100d = word_embeddings("/home/user/Desktop/timesofindiatrainfiles/glove.6B/glove.6B.100d.txt")

In [7]:
### Create sequence
first_1000 = df['sentence']
label_list = df['etype']
vocabulary_size = 20000
tokenizer = Tokenizer(num_words= vocabulary_size)
tokenizer.fit_on_texts(first_1000)
def create_sequences_data(pad_len, seq_data=first_1000):
    sequences = tokenizer.texts_to_sequences(first_1000)
    data = pad_sequences(sequences, maxlen=200)
    return data
data = create_sequences_data(200)

In [8]:
def create_embeddings(word_vector, embedding_dim):
    embedding_matrix = np.zeros((vocabulary_size, embedding_dim))
    for word, index in tokenizer.word_index.items():
        if index > vocabulary_size - 1:
            break
        else:
            embedding_vector = word_vector.get(word)
            if embedding_vector is not None:
                embedding_matrix[index] = embedding_vector
    return embedding_matrix

In [9]:
embedding_matrix_100 = create_embeddings(glove_100d, 100)

In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data, label_list, test_size=0.33, random_state=42)

In [11]:
def conv1d_lstm(embedding_matrix, embedding_dim):
    model_glove = Sequential()
    model_glove.add(Embedding(vocabulary_size, embedding_dim, input_length=200, weights=[embedding_matrix], trainable=False))
    model_glove.add(Dropout(0.2))
    model_glove.add(Conv1D(64, 5, activation='relu'))
    model_glove.add(MaxPooling1D(pool_size=4))
    model_glove.add(LSTM(200))
    model_glove.add(Dense(1, activation='sigmoid'))
    model_glove.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model_glove

In [12]:
def CNN(embedding_matrix, embedding_dim):
    model = Sequential()
    model.add(Embedding(vocabulary_size, embedding_dim, weights=[embedding_matrix], input_length=200, trainable=False))
    model.add(Conv1D(64, 7, activation='relu', padding='same'))
    model.add(MaxPooling1D(2))
    model.add(Conv1D(64, 7, activation='relu', padding='same'))
    model.add(GlobalMaxPooling1D())
    model.add(Dropout(0.5))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [22]:
def LSTM_model(embedding_matrix, embedding_dim):
    model = Sequential()
    model.add(Embedding(vocabulary_size, embedding_dim, weights=[embedding_matrix], input_length=200, trainable=False))
    model.add(Dropout(0.2))
    model.add(LSTM(200))
    model.add(Dense(1,activation='sigmoid'))
    model.compile(loss = 'binary_crossentropy', optimizer='adam',metrics = ['accuracy'])
    return model

In [23]:
model_lstm_100 = LSTM_model(embedding_matrix_100, 100)
model_lstm_100.fit(X_train, np.array(y_train), validation_split=0.0, batch_size=32, epochs=8)

Epoch 1/8
886/886 [==============================] - 11s 13ms/step - loss: 0.5542 - acc: 0.7336
Epoch 2/8
886/886 [==============================] - 10s 11ms/step - loss: 0.4566 - acc: 0.7810
Epoch 3/8
886/886 [==============================] - 10s 11ms/step - loss: 0.4336 - acc: 0.8014
Epoch 4/8
886/886 [==============================] - 10s 11ms/step - loss: 0.3925 - acc: 0.8273
Epoch 5/8
886/886 [==============================] - 10s 11ms/step - loss: 0.3632 - acc: 0.8420
Epoch 6/8
886/886 [==============================] - 10s 11ms/step - loss: 0.3599 - acc: 0.8375
Epoch 7/8
886/886 [==============================] - 9s 10ms/step - loss: 0.3366 - acc: 0.8533
Epoch 8/8
886/886 [==============================] - 9s 11ms/step - loss: 0.3214 - acc: 0.8476


In [15]:
model_cnn_100 = CNN(embedding_matrix_100, 100)
model_cnn_100.fit(X_train, np.array(y_train), validation_split=0.0, batch_size=32, epochs=8)

Epoch 1/8
886/886 [==============================] - 2s 2ms/step - loss: 0.6132 - acc: 0.7077
Epoch 2/8
886/886 [==============================] - 2s 2ms/step - loss: 0.5154 - acc: 0.7573
Epoch 3/8
 96/886 [==>...........................] - ETA: 1s - loss: 0.5054 - acc: 0.7292

KeyboardInterrupt: 

In [20]:
model_conv1d_lstm_100 = conv1d_lstm(embedding_matrix_100, 100)
model_conv1d_lstm_100.fit(X_train, np.array(y_train), validation_split=0.0, batch_size=32, epochs=4)

TypeError: LSTM() missing 1 required positional argument: 'embedding_dim'

In [25]:
def test_model(model, name):
    print(name + " Statistics :")
    
    prediction = model.predict(X_test)
    prediction = [1 if pred>0.5 else 0 for pred in prediction]

    from sklearn.metrics import accuracy_score
    from sklearn.metrics import f1_score
    from sklearn.metrics import precision_score
    from sklearn.metrics import recall_score

    print("F1 Score is : " + str(f1_score(y_test, prediction)))
    print("Accuracy Score is : " + str(accuracy_score(y_test, prediction)))
    print("Precision Score is : " + str(precision_score(y_test, prediction)))
    print("Recall Score is : " + str(recall_score(y_test, prediction)))

In [27]:
test_model(model_cnn_100, "model_cnn_100")
print("")
test_model(model_conv1d_lstm_100, "model_conv1d_lstm_100")
print("")
test_model(model_lstm_100, "model_lstm_100")

model_cnn_100 Statistics :
F1 Score is : 0.30864197530864196
Accuracy Score is : 0.7437070938215103
Precision Score is : 0.8620689655172413
Recall Score is : 0.18796992481203006


model_lstm_100 Statistics :
F1 Score is : 0.4583333333333333
Accuracy Score is : 0.7620137299771167
Precision Score is : 0.7457627118644068
Recall Score is : 0.3308270676691729


In [177]:
model_glove = create_model_Conv1d_LSTM(embedding_matrix_300, 300)

In [165]:
model_glove.fit(X_train, np.array(y_train), validation_split=0.0, batch_size=8, epochs=8)

Epoch 1/4
886/886 [==============================] - 12s 14ms/step - loss: 0.5636 - acc: 0.7336
Epoch 2/4
886/886 [==============================] - 8s 9ms/step - loss: 0.3890 - acc: 0.8296
Epoch 3/4
886/886 [==============================] - 8s 9ms/step - loss: 0.3025 - acc: 0.8736
Epoch 4/4
886/886 [==============================] - 8s 9ms/step - loss: 0.1221 - acc: 0.9639


In [201]:
prediction = pipeline.predict(X_test)
prediction = [1 if pred>0.5 else 0 for pred in prediction]

In [218]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

print(f1_score(y_test, prediction))
print(accuracy_score(y_test, prediction))
print(precision_score(y_test, prediction))
print(recall_score(y_test, prediction))

0.6311111111111111
0.8100686498855835
0.7717391304347826
0.5338345864661654


In [217]:
f1_score(y_test, prediction)

0.6311111111111111